In [1]:
# import salabim_plus as sim_plus
import salabim_plus_raw as sim_plus
import salabim as sim
import part_a, part_b, part_c
import misc_tools
import datetime

now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
runtime = 10080
 
with open('data/output_'+now+'.txt', 'w') as out:
    
    # step 1: create the simulation environment
#     env = sim_plus.EnvironmentPlus(trace=out)
    env = sim_plus.Environment(trace=out)
    
    # step 2: create the machines in the simulation environment
    assembly_bench_1 = sim_plus.Machine(var_name='assembly_bench_1', env=env)
    assembly_bench_2 = sim_plus.Machine(var_name='assembly_bench_2', env=env)
    
    machine_1 = sim_plus.Machine(var_name='machine_1', env=env)
    machine_2 = sim_plus.Machine(var_name='machine_2', env=env)
    machine_3 = sim_plus.Machine(var_name='machine_3', env=env)
    
    # step 3: make machine groupings for common processes
    assembly_bench = sim_plus.MachineGroup(var_name='assembly_bench', env=env,
                                           machines=[assembly_bench_1,
                                                     assembly_bench_2])
    common_process = sim_plus.MachineGroup(var_name='common_process', env=env,
                                           machines=[machine_1, machine_3])
    
    # step 4: create the workers in the simulation environment
    assembler = sim_plus.Worker(var_name='assembler', env=env, capacity=2)
    technician = sim_plus.Worker(var_name='technician', env=env, capacity=2)
    
    # step 5: make the shift schedules for the workers
    shift_schedule = misc_tools.make_shifts(shift_duration=8*60,
                                            off_days=['saturday','sunday'])
    
    assembler_shift = sim_plus.ShiftController(worker=assembler,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    technician_shift = sim_plus.ShiftController(worker=technician,
                                               env=env,
                                               start_time=480,
                                               shifts=shift_schedule, 
                                               shift_type='pattern' )
    
    # step 6: point to the step creation function in each entity .py file
    part_a_steps_func = part_a.create_routing
    part_b_steps_func = part_b.create_routing
    part_c_steps_func = part_c.create_routing
    
    # step 7: make all of the generators that determine when entities are made
    part_a_gen = sim_plus.EntityGenerator(var_name='part_a', 
                                         steps_func=part_a_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    part_b_gen = sim_plus.EntityGenerator(var_name='part_b', 
                                         steps_func=part_b_steps_func, 
                                         env=env,
                                         arrival_type='ordered')
    part_c_gen = sim_plus.EntityGenerator(var_name='part_c', 
                                          steps_func=part_c_steps_func,
                                          env=env,
                                          arrival_type='inv_based', 
                                          cut_queue=True, 
                                          inv_level=3)
    
    # step 8: create all of the inventory locations in the simulation environment
    part_a_kanban = sim_plus.Kanban(var_name='part_a', env=env, 
#                                    kanban_attr=part_a.create_kanban_attrs(env.objs))
                                   kanban_attr=part_a.create_kanban_attrs(env._env_objs))
    part_b_kanban = sim_plus.Kanban(var_name='part_b', env=env, 
#                                    kanban_attr=part_b.create_kanban_attrs(env.objs))
                                   kanban_attr=part_b.create_kanban_attrs(env._env_objs))
    
    part_c_storage = sim_plus.Storage(var_name='part_c', env=env)
    scrap = sim_plus.Storage(var_name='scrap', env=env)
    
    # step 9: connect any entity boms and main_exits to its entity generator 
#     part_c_gen.bom = part_c.get_bom(env=env.objs)
    part_c_gen.bom = part_c.get_bom(env=env._env_objs)
    part_a_gen.main_exit = part_a_kanban
    part_b_gen.main_exit = part_b_kanban
    
    # step 10: activate the controlling components within the simulation environment
    assembler_shift.activate(process='work')
    technician_shift.activate(process='work')
    
    part_a_gen.activate(process='arrive')
    part_b_gen.activate(process='arrive')
    part_c_gen.activate(process='arrive')
    
    # step 11: execute the simulation
    env.run(till=runtime)

In [3]:
!pip freeze

asn1crypto==1.3.0
attrs==19.3.0
backcall==0.1.0
beautifulsoup4==4.8.2
bleach==3.1.0
certifi==2019.11.28
cffi==1.14.0
chardet==3.0.4
colorama==0.4.3
cryptography==2.8
cycler==0.10.0
decorator==4.4.1
deepdiff==4.0.9
defusedxml==0.6.0
descartes==1.1.0
entrypoints==0.3
hermes==1.5.2
idna==2.8
importlib-metadata==1.3.0
ipykernel==5.1.3
ipython==7.10.2
ipython-genutils==0.2.0
ipywidgets==7.5.1
jedi==0.15.1
Jinja2==2.10.3
jira==2.0.0
joblib==0.14.1
json5==0.8.5
jsonpickle==1.2
jsonschema==3.2.0
jupyter-client==5.3.4
jupyter-core==4.6.1
jupyterlab==1.2.4
jupyterlab-server==1.0.6
kiwisolver==1.1.0
MarkupSafe==1.1.1
matplotlib==3.1.1
mistune==0.8.4
mizani==0.6.0
mkl-fft==1.0.15
mkl-random==1.1.0
mkl-service==2.3.0
more-itertools==8.0.2
nbconvert==5.6.1
nbformat==4.4.0
notebook==6.0.2
ntlm-auth==1.4.0
numpy==1.18.1
oauthlib==3.1.0
olefile==0.46
ordered-set==3.1.1
palettable==3.3.0
pandas==1.0.1
pandocfilters==1.4.2
parso==0.5.2
patsy==0.5.1
pbr==5.4.4
pickleshare==0.7.5
Pillow==7.0.0
plotly==4.4.